In [55]:
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

In [69]:
import pandas as pd
train = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting = 3)
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting = 3)

In [ ]:
num_features = 300 
min_word_count = 40 
num_workers = 4
context = 10 
downsampling = 1e-3 

In [71]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    review_text = BeautifulSoup(review,"lxml").get_text()
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    words = review_text.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return(words)

In [72]:
type(model.wv.vectors)

numpy.ndarray

In [73]:
print(model.wv.vectors.shape)

(16490, 300)


In [74]:
print(model.wv["flower"])

[-5.06587438e-02 -1.30716218e-02 -2.69954707e-02  6.33645728e-02
 -6.51245043e-02 -6.32605106e-02 -4.81012336e-04 -1.11888098e-02
 -7.76432529e-02 -4.86407056e-02  9.42130908e-02  5.38372397e-02
  5.48828393e-02  7.37902569e-03 -1.48566850e-02  2.60501076e-02
 -5.07104732e-02  1.32804230e-01  2.36674372e-04 -6.28971010e-02
  5.41811902e-03  1.13381026e-02 -9.77564007e-02 -2.56607942e-02
 -4.20020781e-02  6.11054152e-02  7.67687634e-02  2.49413797e-03
  2.04210775e-03 -1.03719108e-01  1.49613708e-01  4.36168276e-02
  4.21950445e-02  6.13381118e-02 -3.76603231e-02 -2.22360268e-02
 -6.94532022e-02 -1.18287876e-01  3.16194966e-02  6.24483339e-02
 -7.27119520e-02  4.38906811e-02  3.55872139e-02 -7.94034302e-02
  5.18171936e-02 -4.87119611e-03  6.46155477e-02  5.70900179e-02
 -9.54613760e-02 -3.84350047e-02  6.64945915e-02  6.05421513e-03
 -2.43431292e-02  9.96646360e-02 -2.78943069e-02  1.27462600e-03
 -7.82098528e-03  1.18875699e-02  2.63399892e-02  3.64927538e-02
  1.58986285e-01  4.92288

In [75]:
import numpy as np

In [81]:
def makeFeatureVec(words, model, num_features):

    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0
    
    index2word_set = set(model.wv.index2word)
    
    for word in words:
        if word in index2word_set:
            nwords = nwords+1.
            featureVec = np.add(featureVec, model[word])
    
    featureVec = np.divide(featureVec, nwords)
    return featureVec
    

In [82]:
def getAvgFeatureVecs(reviews, model, num_features):
    
    counter = 0
    counter = int(counter)
    
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    
    for review in reviews:
        if counter%1000. == 0.:
            print("Review %d of %d" % (counter, len(reviews)))
        
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter = counter + 1
        
    return reviewFeatureVecs

In [83]:
print ("Creating average feature vecs for Training reviews")
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

Creating average feature vecs for Training reviews
Review 0 of 25000


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide


Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000


In [84]:
print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Creating average feature vecs for test reviews
Review 0 of 25000


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000


In [95]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Imputer

forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
trainDataVecs = Imputer().fit_transform(trainDataVecs)

forest = forest.fit( trainDataVecs, train["sentiment"] )

Fitting a random forest to labeled training data...


In [97]:
testDataVecs = Imputer().fit_transform(testDataVecs)
result = forest.predict( testDataVecs )

output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

In [98]:
from sklearn.cluster import KMeans
import time
start = time.time() 

In [103]:
word_vectors = model.wv.vectors
num_clusters = int(word_vectors.shape[0] / 5)

In [104]:
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

In [106]:
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

Time taken for K Means clustering:  1180.7468872070312 seconds.


In [108]:
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [116]:
for cluster in range(0,10):
    
    print("\nCluster %d" % cluster)
   
    words = []
    for i in range(0,len(list(word_centroid_map.values()))):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map)[i])
    print(words)


Cluster 0
['bent']

Cluster 1
['gleefully', 'glee', 'vomiting', 'leering', 'teasing', 'orville']

Cluster 2
['olivia', 'havilland']

Cluster 3
['gunslinger', 'scrimm', 'lawman', 'wily']

Cluster 4
['diversity']

Cluster 5
['backs', 'throats', 'butts', 'noses', 'beds', 'pockets', 'ropes', 'tongues', 'necks']

Cluster 6
['bond', 'stewart', 'cagney', 'cameron', 'mason', 'garner', 'belushi', 'coburn', 'gleason', 'caan', 'russo', 'gandolfini', 'brolin', 'toback', 'spader', 'caviezel', 'duval', 'howe']

Cluster 7
['romania', 'denmark', 'netherlands', 'serbia']

Cluster 8
['maintains', 'maintained', 'entertains', 'retains', 'balances']

Cluster 9
['hopkins', 'dern', 'perkins', 'viggo', 'mortensen', 'elisha', 'christensen']


In [117]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    num_centroids = max( word_centroid_map.values() ) + 1
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    return bag_of_centroids

In [118]:
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [119]:
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [121]:
forest = RandomForestClassifier(n_estimators = 100)

print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
